# ADHD200: End-to-end pipeline (BIDS → fMRIPrep → ROI time series)

Этот ноутбук запускает:
1) **fMRIPrep** через `fmriprep-docker` (контейнер Docker) для препроцессинга fMRI.
2) Экстракцию ROI-таймсерий по выбранному атласу. В данной работе используется атлас из библиотеки `nilearn`: **Schaefer**, 17 сетей, 400 ROI, разрешение 2 мм.

### 1. Импорт библиотек и определение параметров

In [ ]:
%pip install fmriprep-docker

In [ ]:
# Необходимые библиотеки
import os, glob
import subprocess
import shlex
import shutil
from pathlib import Path
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
import fmriprep_docker as fmriprep

In [ ]:
# Абсолютный путь к текущему рабочему каталогу
current_dir = os.getcwd()
print(f"Текущий рабочий каталог: {current_dir}")

# Корневая папка текущего пользователя
home_dir = str(Path.home())
print(f"Корневая папка пользователя: {home_dir}")

# Пути к существующим папкам
BIDS_SORTED_DIR = current_dir + "../SortedRawDataBIDS" # существующая папка с отсортированными BIDS-данными
BIDS_DIR_TEEN = BIDS_SORTED_DIR + "/cohort_teen_participants" # папка с BIDS-данными подростков
BIDS_DIR_ADULT = BIDS_SORTED_DIR + "/cohort_adult_participants" # папка с BIDS-данными взрослых

Текущий рабочий каталог: /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/Материалы/ADHD200
Корневая папка пользователя: /Users/alexey.stafeev


Из библиотеки `nilearn` используются:
- `nilearn.datasets` для обработки атласов (Schaefer, 2018)

In [4]:
# %pip install nilearn
from nilearn.datasets import fetch_atlas_schaefer_2018

atlas = fetch_atlas_schaefer_2018(n_rois=400, yeo_networks=17, resolution_mm=2)
DEFAULT_ATLAS_IMG = atlas['maps']
DEFAULT_ATLAS_LABELS = atlas['labels']
print('Atlas NIfTI:', DEFAULT_ATLAS_IMG)

# Labels - в формате списка: ['1', '2', ..., '400', '17Networks_1', ..., '17Networks_17']
# Сохраним их в текстовый файл
LABELS_OUT_TXT = current_dir + "/schaefer400_17Networks_labels.txt"
with open(LABELS_OUT_TXT, "w") as f:
    for label in DEFAULT_ATLAS_LABELS:
        f.write(label + "\n")
print('Labels TXT:', LABELS_OUT_TXT)

[fetch_atlas_schaefer_2018] Dataset found in /Users/alexey.stafeev/nilearn_data/schaefer_2018

Atlas NIfTI: /Users/alexey.stafeev/nilearn_data/schaefer_2018/Schaefer2018_400Parcels_17Networks_order_FSLMNI152_2mm.nii.gz
Labels TXT: /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/Материалы/ADHD200/schaefer400_17Networks_labels.txt


In [5]:
# Создадим директории для сохранения результатов обработки fmriprep

print(f"\n--- 1.1 Создание директорий для сохранения результатов обработки fmriprep ---")

# Корневая директория для сохранения результатов обработки fmriprep (derivatives)
DERIV_DIR_ROOT = current_dir + "/FMRIPrepData"

# Если папка существует, удалим её и создадим заново
if os.path.exists(DERIV_DIR_ROOT):
    print(f"Папка {DERIV_DIR_ROOT} уже существует. Удаляем и создаём заново.")
    shutil.rmtree(DERIV_DIR_ROOT)
    
os.makedirs(DERIV_DIR_ROOT, exist_ok=True)
print(f"Создана или уже существует папка {DERIV_DIR_ROOT}.")

# Создадим папки для подростков и взрослых
DERIV_DIR_TEEN = DERIV_DIR_ROOT + "/cohort_teen_participants"
os.makedirs(DERIV_DIR_TEEN, exist_ok=True)
print(f"Создана или уже существует папка {DERIV_DIR_TEEN}.")

DERIV_DIR_ADULT = DERIV_DIR_ROOT + "/cohort_adult_participants"
os.makedirs(DERIV_DIR_ADULT, exist_ok=True)
print(f"Создана или уже существует папка {DERIV_DIR_ADULT}.")


--- 1.1 Создание директорий для сохранения результатов обработки fmriprep ---
Папка /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/Материалы/ADHD200/FMRIPrepData уже существует. Удаляем и создаём заново.
Создана или уже существует папка /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/Материалы/ADHD200/FMRIPrepData.
Создана или уже существует папка /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/Материалы/ADHD200/FMRIPrepData/cohort_teen_participants.
Создана или уже существует папка /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/Материалы/ADHD200/FMRIPrepData/cohort_adult_participants.


In [6]:
# Создадим временную папку для обработки fmriprep
FMRIPREP_WORK_DIR = home_dir + "/fmriprep_work"
if os.path.exists(FMRIPREP_WORK_DIR):
    print(f"Папка {FMRIPREP_WORK_DIR} уже существует. Удаляем и создаём заново.")
    shutil.rmtree(FMRIPREP_WORK_DIR)

os.makedirs(FMRIPREP_WORK_DIR, exist_ok=True)
print(f"Создана или уже существует папка {FMRIPREP_WORK_DIR}.")

Папка /Users/alexey.stafeev/fmriprep_work уже существует. Удаляем и создаём заново.
Создана или уже существует папка /Users/alexey.stafeev/fmriprep_work.


In [7]:
# Создадим файлы с участниками для каждой когорты
import pandas as pd

print(f"\n--- Создание файлов с участниками для каждой когорты ---")
# Файл с участниками для подростков
PARTICIPANTS_ID_FILE_TEEN = DERIV_DIR_TEEN + "/participants.txt"
teen_tsv = BIDS_DIR_TEEN + "/participants.tsv"
df_teen = pd.read_csv(teen_tsv, sep='\t')

# Извлекаем данные столбца participant_id, удаляем префикс sub- и сохраняем в текстовый файл
df_teen['participant_id_no_sub'] = df_teen['participant_id'].str.replace('sub-', '')
df_teen[['participant_id_no_sub']].to_csv(PARTICIPANTS_ID_FILE_TEEN, index=False, header=False)
print(f"Файл с участниками для подростков сохранён в {PARTICIPANTS_ID_FILE_TEEN}")

#  Извлекаем 1 участника для теста в список
PARTICIPANTS_ID_TEEN = df_teen['participant_id_no_sub'].tolist()[:1]
print("Участники для теста (подростки):", PARTICIPANTS_ID_TEEN)

# Файл с участниками для взрослых
PARTICIPANTS_ID_FILE_ADULT = DERIV_DIR_ADULT + "/participants.txt"
adult_tsv = BIDS_DIR_ADULT + "/participants.tsv"
df_adult = pd.read_csv(adult_tsv, sep='\t')

# Извлекаем данные столбца participant_id, удаляем префикс sub- и сохраняем в текстовый файл
df_adult['participant_id_no_sub'] = df_adult['participant_id'].str.replace('sub-', '')
df_adult[['participant_id_no_sub']].to_csv(PARTICIPANTS_ID_FILE_ADULT, index=False, header=False)
print(f"Файл с участниками для взрослых сохранён в {PARTICIPANTS_ID_FILE_ADULT}")

#  Извлекаем 1 участника для теста в список
PARTICIPANTS_ID_ADULT = df_adult['participant_id_no_sub'].tolist()[:1]
print("Участники для теста (взрослые):", PARTICIPANTS_ID_ADULT)


--- Создание файлов с участниками для каждой когорты ---
Файл с участниками для подростков сохранён в /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/Материалы/ADHD200/FMRIPrepData/cohort_teen_participants/participants.txt
Участники для теста (подростки): ['kki2026113']
Файл с участниками для взрослых сохранён в /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/Материалы/ADHD200/FMRIPrepData/cohort_adult_participants/participants.txt
Участники для теста (взрослые): ['neuroimage1125505']


In [8]:
# Создадим директорию для сохранения производных данных (таймсерий)

print(f"\n--- 1.2 Создание структуры директорий для сохранения производных данных (таймсерий) ---")

ROI_TIMESERIES_ROOT = current_dir + "/ROITimeseries"
if os.path.exists(ROI_TIMESERIES_ROOT):
    print(f"Папка {ROI_TIMESERIES_ROOT} уже существует. Удаляем и создаём заново.")
    shutil.rmtree(ROI_TIMESERIES_ROOT)

# Создадим папки для подростков и взрослых
TS_OUT_TEEN = ROI_TIMESERIES_ROOT + "/cohort_teen_participants"
os.makedirs(TS_OUT_TEEN, exist_ok=True)
print(f"Создана или уже существует папка {TS_OUT_TEEN}.")

TS_OUT_ADULT = ROI_TIMESERIES_ROOT + "/cohort_adult_participants"
os.makedirs(TS_OUT_ADULT, exist_ok=True)
print(f"Создана или уже существует папка {TS_OUT_ADULT}.")


--- 1.2 Создание структуры директорий для сохранения производных данных (таймсерий) ---
Папка /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/Материалы/ADHD200/ROITimeseries уже существует. Удаляем и создаём заново.
Создана или уже существует папка /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/Материалы/ADHD200/ROITimeseries/cohort_teen_participants.
Создана или уже существует папка /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/Материалы/ADHD200/ROITimeseries/cohort_adult_participants.


In [9]:
# Параметры для обработки

# Параметры для fmriprep
WORK_DIR = FMRIPREP_WORK_DIR # временная папка для обработки (желательно на SSD)
FS_LICENSE = current_dir + "/license.txt"
OUTPUT_SPACES = "MNI152NLin2009cAsym:res-2"
NTHREADS = 8
OMP_NTHREADS = 8
MEM_MB = 32000

# Параметры для извлечения таймсерий
ATLAS_IMG = DEFAULT_ATLAS_IMG
ATLAS_LABELS = LABELS_OUT_TXT
FD_THRESH = 0.5
N_ACOMPCOR = 5
USE_GSR = 0
LOW_PASS = 0.08
HIGH_PASS = 0.008
MIN_VOLS = 120

PARTICIPANTS_TEEN = PARTICIPANTS_ID_TEEN # список ID участников (используется для теста)
PARTICIPANTS_ADULT = PARTICIPANTS_ID_ADULT # список ID участников (используется для теста)
# Или путь к файлу с ID участников (один ID на строку)
PARTICIPANTS_FILE_TEEN = PARTICIPANTS_ID_FILE_TEEN
PARTICIPANTS_FILE_ADULT = PARTICIPANTS_ID_FILE_ADULT
# (если указан, то игнорируется список PARTICIPANTS)
# Или None, чтобы обработать всех участников в BIDS_DIR

In [10]:
print("BIDS_DIR_TEEN exists:", Path(BIDS_DIR_TEEN).exists())
print("BIDS_DIR_ADULT exists:", Path(BIDS_DIR_ADULT).exists())

print("DERIV_DIR_TEEN exists:", Path(DERIV_DIR_TEEN).exists())
if not Path(DERIV_DIR_TEEN).exists():
    os.makedirs(DERIV_DIR_TEEN, exist_ok=True)
    print(f"Каталог {DERIV_DIR_TEEN} создан.")

print("DERIV_DIR_ADULD exists:", Path(DERIV_DIR_ADULT).exists())
if not Path(DERIV_DIR_ADULT).exists():
    os.makedirs(DERIV_DIR_ADULT, exist_ok=True)
    print(f"Каталог {DERIV_DIR_ADULT} создан.")

print("WORK_DIR exists:", Path(WORK_DIR).exists())
# Проверим, что в WORK_DIR есть права на запись
if not os.access(WORK_DIR, os.W_OK):
    raise PermissionError(f"Нет прав на запись в WORK_DIR: {WORK_DIR}")
else:
    print(f"Есть права на запись в WORK_DIR: {WORK_DIR}")

print("FS_LICENSE exists:", Path(FS_LICENSE).exists())
print("ATLAS_IMG exists:", Path(ATLAS_IMG).exists())
print("ATLAS_LABELS exists:", Path(ATLAS_LABELS).exists())

BIDS_DIR_TEEN exists: True
BIDS_DIR_ADULT exists: True
DERIV_DIR_TEEN exists: True
DERIV_DIR_ADULD exists: True
WORK_DIR exists: True
Есть права на запись в WORK_DIR: /Users/alexey.stafeev/fmriprep_work
FS_LICENSE exists: True
ATLAS_IMG exists: True
ATLAS_LABELS exists: True


In [11]:
# Проверка FreeSurfer лицензии
print("FS_LICENSE env:", os.environ.get("FS_LICENSE"))
print("FS_LICENSE (из параметров):", FS_LICENSE)
print("Лицензия существует?", Path(FS_LICENSE).exists())
if Path(FS_LICENSE).exists():
    print("Размер license.txt:", Path(FS_LICENSE).stat().st_size, "байт")
else:
    raise FileNotFoundError(f"Файл лицензии FreeSurfer не найден: {FS_LICENSE}")

FS_LICENSE env: None
FS_LICENSE (из параметров): /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/Материалы/ADHD200/license.txt
Лицензия существует? True
Размер license.txt: 133 байт


In [46]:
def run_command(cmd_list):
    # потоковый вывод
    proc = subprocess.Popen(cmd_list, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
    for line in proc.stdout:
        print(line, end="")
    return proc.wait()

### 2. Обработка BIDS-данных с помощью fMRIPrep - извлечение деривативов

In [ ]:
# Обёртка над fmriprep для обработки BIDS-данных и извлечения таймсерий
def process_fmriprep(bids_dir, deriv_dir, participants=None, work_dir=None,
                     fs_license=None, output_spaces="MNI152NLin2009cAsym:res-2",
                     nthreads=8, omp_nthreads=8, mem_mb=32000,
                     skip_bids_validation=True, extra_container_opts=None,
                     fs_no_recon_all=False, no_tty=True):
    """ Запускает fmriprep для указанной папки BIDS-данных и сохраняет результаты в deriv_dir.
        Если participants указаны, обрабатывает только этих участников.
        Иначе обрабатывает всех участников в bids_dir.
    """
    
    # нормализуем пути
    bids_dir   = str(Path(bids_dir).expanduser().resolve())
    deriv_dir  = str(Path(deriv_dir).expanduser().resolve())
    work_dir   = str(Path(work_dir).expanduser().resolve())
    fs_license = str(Path(fs_license).expanduser().resolve())

    # participants: список имён без 'sub-'
    if participants is None:
        part_list = []
    elif isinstance(participants, str) and os.path.isfile(participants):
        with open(participants, 'r') as f:
            part_list = [ln.strip() for ln in f if ln.strip()]
    elif isinstance(participants, list):
        part_list = participants
    else:
        raise ValueError("participants: list | path-to-file | None")

    # найти обёртку
    fmriprep_local = shutil.which("fmriprep-docker") or shutil.which(str(Path.home()/".local/bin/fmriprep-docker"))
    if not fmriprep_local:
        raise EnvironmentError("fmriprep-docker не найден в PATH.")

    cmd = [
        fmriprep_local,
        bids_dir,
        deriv_dir,
        "participant",                      # ВАЖНО!
        "--fs-license-file", fs_license,
        "--work-dir", work_dir,
        "--output-spaces", output_spaces,
        "--omp-nthreads", str(omp_nthreads),
        "--nthreads", str(nthreads),
        "--mem-mb", str(int(mem_mb)),
        "--verbose",
    ]
    if fs_no_recon_all:
        cmd.append("--fs-no-recon-all")
    if no_tty:
        cmd.append("--no-tty")
    if skip_bids_validation:
        cmd.append("--skip-bids-validation")
    if part_list:
        cmd += ["--participant-label"] + part_list
    if extra_container_opts:
        cmd += ["--container-options", extra_container_opts]

    print("Будет выполнена команда:\n", " ".join(shlex.quote(x) for x in cmd))
    return run_command(cmd)

In [53]:
# Тестовый запуск fmriprep для подростков (с использованием списка с единственным участником)
print(f"\n--- 2.0 Запуск fmriprep для когорты подростков из {BIDS_DIR_TEEN} ---")
ret_fmriprep_teen = process_fmriprep(BIDS_DIR_TEEN, DERIV_DIR_TEEN,
                                   participants=PARTICIPANTS_TEEN,
                                   work_dir=WORK_DIR, fs_license=FS_LICENSE,
                                   output_spaces=OUTPUT_SPACES,
                                   nthreads=NTHREADS, omp_nthreads=OMP_NTHREADS, mem_mb=MEM_MB)
print("\nКод возврата:", ret_fmriprep_teen)


--- 2.0 Запуск fmriprep для когорты подростков из /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/Материалы/ADHD200/SortedRawDataBIDS/cohort_teen_participants ---
Будет выполнена команда:
 /Users/alexey.stafeev/.local/bin/fmriprep-docker '/Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/Материалы/ADHD200/SortedRawDataBIDS/cohort_teen_participants' '/Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/Материалы/ADHD200/FMRIPrepData/cohort_teen_participants' participant --fs-license-file '/Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/Материалы/ADHD200/license.txt' --work-dir /Users/alexey.stafeev/fmriprep_work --output-spaces MNI152NLin2009cAsym:res-2 --omp-nthreads 8 --nthreads 8 --mem-mb 32000 --verbose --no-tty --skip-bids-validation --participant-label kki2026113
RUNNING: docker run --rm -e DOCKER_VERSION_8395080871=28.0.4 -v /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТ

KeyboardInterrupt: 

In [0]:
print(f"\n--- 2.1 Запуск fmriprep для BIDS-данных возрастной когорты подростков в {BIDS_DIR_TEEN} ---")
print(f"Результаты будут сохранены в {DERIV_DIR_TEEN}")

ret_fmriprep_teen = process_fmriprep(
    bids_dir=BIDS_DIR_TEEN,
    deriv_dir=DERIV_DIR_TEEN,
    participants=PARTICIPANTS_FILE_TEEN, # или PARTICIPANTS_TEEN
    work_dir=WORK_DIR,
    fs_license=FS_LICENSE,
    output_spaces=OUTPUT_SPACES,
    nthreads=NTHREADS,
    omp_nthreads=OMP_NTHREADS,
    mem_mb=MEM_MB
)

print("\n=== fMRIPrep завершён. Код возврата:", ret_fmriprep_teen, "===")

In [0]:
print(f"\n--- 2.2 Запуск fmriprep для BIDS-данных возрастной когорты молодых взрослых в {BIDS_DIR_TEEN} ---")
print(f"Результаты будут сохранены в {DERIV_DIR_TEEN}")

ret_fmriprep_adult = process_fmriprep(
    bids_dir=BIDS_DIR_TEEN,
    deriv_dir=DERIV_DIR_TEEN,
    participants=PARTICIPANTS_FILE_ADULT, # или PARTICIPANTS_ADULT
    work_dir=WORK_DIR,
    fs_license=FS_LICENSE,
    output_spaces=OUTPUT_SPACES,
    nthreads=NTHREADS,
    omp_nthreads=OMP_NTHREADS,
    mem_mb=MEM_MB
)

print("\n=== fMRIPrep завершён. Код возврата:", ret_fmriprep_adult, "===")

### 3. Проверка деривативов после обработки BIDS с помощью fMRIPrep

In [41]:
# === Проверка: наличие поверхностных/объёмных деривативов ===

# Проверим деривативы fMRIPrep (объёмные)
def check_derivatives(deriv_dir):
    """Проверяет наличие деривативов fMRIPrep в указанной директории."""
    fprep_root = Path(deriv_dir) / "fmriprep"
    print("\nДиректория fMRIPrep существует?", fprep_root.exists())
    if fprep_root.exists():
        subs = sorted([p.name for p in fprep_root.glob("sub-*") if p.is_dir()])
        print("Найдено субъектов в fmriprep:", len(subs))
        if subs:
            sample = fprep_root / subs[0]
            func_hits = list(sample.rglob("*desc-preproc_bold.nii.gz"))
            conf_hits = list(sample.rglob("*desc-confounds_timeseries.tsv"))
            print("Пример субъекта:", sample)
            print("BOLD препроцессированных файлов:", len(func_hits))
            print("Confounds tsv:", len(conf_hits))
            if func_hits[:3]:
                for p in func_hits[:3]:
                    print(" -", p)
    else:
        print("\nПапка деривативов ещё не создана или пути не заполнены. Требуется перезапуск fMRIPrep.")

# Проверим директорию freesurfer (поверхности), если recon-all выполнялся
def check_freesurfer(deriv_dir):
    """Проверяет наличие деривативов FreeSurfer в указанной директории."""
    fs_root = Path(deriv_dir) / "freesurfer"
    print("\nДиректория freesurfer существует?", fs_root.exists())
    if fs_root.exists():
        subs_fs = sorted([p.name for p in fs_root.glob("sub-*") if p.is_dir()])
        print("Найдено субъектов в freesurfer:", len(subs_fs))
        if subs_fs:
            sample_fs = fs_root / subs_fs[0]
            lh_pial = sample_fs / "surf" / "lh.pial"
            rh_pial = sample_fs / "surf" / "rh.pial"
            aseg = sample_fs / "mri" / "aseg.mgz"
            print("Пример субъекта FS:", sample_fs)
            print("lh.pial:", lh_pial.exists(), " rh.pial:", rh_pial.exists(), " aseg.mgz:", aseg.exists())
            # Показать несколько surface-файлов
            surf_list = list((sample_fs/"surf").glob("*.pial"))[:5]
            if surf_list:
                print("Примеры surface-файлов:")
                for p in surf_list:
                    print(" -", p)
    else:
        print("\nПапка деривативов ещё не создана или пути не заполнены. Требуется перезапуск fMRIPrep.")

In [ ]:
print(f"\n--- 3.1 Проверка деривативов fMRIPrep для подростков в {DERIV_DIR_TEEN} ---")
check_derivatives(DERIV_DIR_TEEN)
check_freesurfer(DERIV_DIR_TEEN)

In [ ]:
print(f"\n--- 3.2 Проверка деривативов fMRIPrep для взрослых в {DERIV_DIR_ADULT} ---")
check_derivatives(DERIV_DIR_ADULT)
check_freesurfer(DERIV_DIR_ADULT)

### 4. Экстракция ROI-таймсерий (Schaefer, Harvard-Oxford и др.)
В следующей ячейке сохраняется локально скрипт `adhd200_derivs_pipeline.py`, который извлекает таймсерии.

In [0]:
PIPE_PATH = Path("./adhd200_derivs_pipeline.py")
# PIPE_CODE = r"""#!/usr/bin/env python3
# import argparse
# from pathlib import Path
# from typing import Dict, List, Optional, Tuple
# import numpy as np, pandas as pd, nibabel as nib
# from nilearn.maskers import NiftiLabelsMasker
# def find_func_derivs(root: Path, subs: Optional[List[str]] = None):
#     pats = sorted(root.glob("sub-*/*/func/*desc-preproc_bold.nii.gz"))
#     if not pats:
#         pats = sorted(root.glob("sub-*/func/*desc-preproc_bold.nii.gz"))
#     out = []
#     for bold in pats:
#         parts = bold.name.split("_")
#         info = {"bold": bold, "confounds": None, "mask": None, "sub": None, "ses": None, "task": None, "run": None}
#         for p in parts:
#             if p.startswith("sub-"): info["sub"] = p[4:]
#             elif p.startswith("ses-"): info["ses"] = p[4:]
#             elif p.startswith("task-"): info["task"] = p[5:]
#             elif p.startswith("run-"): info["run"] = p[4:]
#         if subs and info["sub"] not in subs: continue
#         conf = bold.parent / bold.name.replace("_desc-preproc_bold.nii.gz", "_desc-confounds_timeseries.tsv")
#         mask = bold.parent / bold.name.replace("_desc-preproc_bold.nii.gz", "_desc-brain_mask.nii.gz")
#         if conf.exists(): info["confounds"] = conf
#         if mask.exists(): info["mask"] = mask
#         out.append(info)
#     return out
# def load_confounds(conf_path: Path, fd_thresh=0.5, n_acompcor=5, use_gsr=False):
#     import pandas as pd, numpy as np
#     cf = pd.read_csv(conf_path, sep='\t')
#     meta = {}
#     fd = cf.get('framewise_displacement')
#     sm = None
#     if fd is not None:
#         bad = fd.fillna(0).values > fd_thresh
#         nss = [c for c in cf.columns if c.startswith('non_steady_state_outlier')]
#         if nss: bad = np.logical_or(bad, (cf[nss].fillna(0).sum(axis=1) > 0).values)
#         good = np.where(~bad)[0]
#         if len(good) > 0: sm = good
#         meta['n_scrubbed'] = int(bad.sum())
#     else:
#         meta['n_scrubbed'] = 0
#     cols = [c for c in ['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z'] if c in cf.columns]
#     cols += [c+'_derivative1' for c in ['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z'] if c+'_derivative1' in cf.columns]
#     acomp = [c for c in cf.columns if c.startswith('a_comp_cor')]
#     acomp = acomp[:n_acompcor] if n_acompcor>0 else []
#     cols += acomp
#     for c in ['white_matter','csf']:
#         if c in cf.columns: cols.append(c)
#     if use_gsr and 'global_signal' in cf.columns: cols.append('global_signal')
#     X = cf[cols].fillna(0).values if cols else None
#     return X, sm, meta
# def guess_tr(bold_img: Path):
#     import nibabel as nib, numpy as np
#     hdr = nib.load(str(bold_img)).header
#     try:
#         tr = float(hdr.get_zooms()[3])
#         return tr if np.isfinite(tr) and tr>0 else None
#     except Exception:
#         return None
# def extract_ts(bold, atlas, mask, confounds, sample_mask, tr, lp, hp):
#     masker = NiftiLabelsMasker(labels_img=str(atlas), mask_img=(str(mask) if mask else None),
#                                standardize='zscore', detrend=True, high_pass=hp, low_pass=lp, t_r=tr,
#                                resampling_target='data')
#     return masker.fit_transform(str(bold), confounds=confounds, sample_mask=sample_mask)
# def main():
#     ap = argparse.ArgumentParser()
#     ap.add_argument('--deriv-root', required=True)
#     ap.add_argument('--atlas-img', required=True)
#     ap.add_argument('--atlas-labels', default=None)
#     ap.add_argument('--participants', default=None)
#     ap.add_argument('--out', required=True)
#     ap.add_argument('--fd-thresh', type=float, default=0.5)
#     ap.add_argument('--n-acompcor', type=int, default=5)
#     ap.add_argument('--use-gsr', type=int, default=0)
#     ap.add_argument('--low-pass', type=float, default=0.08)
#     ap.add_argument('--high-pass', type=float, default=0.008)
#     ap.add_argument('--min-vols', type=int, default=120)
#     args = ap.parse_args()
#     root = Path(args.deriv_root).expanduser().resolve()
#     atlas = Path(args.atlas_img).expanduser().resolve()
#     outd = Path(args.out).expanduser().resolve(); outd.mkdir(parents=True, exist_ok=True)
#     subs = None
#     if args.participants:
#         subs = [ln.strip().replace('sub-','') for ln in Path(args.participants).read_text().splitlines() if ln.strip()]
#     runs = find_func_derivs(root, subs)
#     import pandas as pd, numpy as np
#     meta = []
#     kept = []
#     for r in runs:
#         bold, conf, mask = r['bold'], r['confounds'], r['mask']
#         Xc, sm, m = load_confounds(conf, fd_thresh=args.fd_thresh, n_acompcor=args.n_acompcor, use_gsr=bool(args.use_gsr))
#         tr = guess_tr(bold)
#         ts = extract_ts(bold, atlas, mask, Xc, sm, tr, args.low_pass, args.high_pass)
#         keep = int(ts.shape[0] >= args.min_vols)
#         sub_dir = outd / f"sub-{r['sub']}" / (f"ses-{r['ses']}" if r['ses'] else 'ses-NA')
#         sub_dir.mkdir(parents=True, exist_ok=True)
#         name = f"sub-{r['sub']}_{('ses-'+r['ses']+'_') if r['ses'] else ''}task-{r['task'] or 'rest'}_{('run-'+r['run']+'_') if r['run'] else ''}timeseries.tsv"
#         (sub_dir / name).write_text(pd.DataFrame(ts).to_csv(sep='\t', index=False))
#         np.save(sub_dir / name.replace('.tsv','.npy'), ts)
#         meta.append({'subject': r['sub'], 'session': r['ses'] or '', 'run': r['run'] or '', 'task': r['task'] or 'rest',
#                      'bold': str(bold), 'confounds': str(conf), 'mask': str(mask) if mask else '',
#                      'tr': tr, 'n_volumes_after_scrub': int(ts.shape[0]), 'kept': keep, 'n_confounds': 0, 'n_scrubbed': m.get('n_scrubbed', 0)})
#         if keep: kept.append(ts)
#     np.savez(outd / 'timeseries_agg.npz', *kept)
#     pd.DataFrame(meta).to_csv(outd / 'processing_manifest.tsv', sep='\t', index=False)
# if __name__ == '__main__':
#     main()
# """
# PIPE_PATH.write_text(PIPE_CODE, encoding="utf-8")
print("Скрипт сохранён:", PIPE_PATH)

In [ ]:
# Небольшая обёртка над adhd200_derivs_pipeline.py

def run_pipeline(deriv_root, atlas_img, atlas_labels, out, participants=None, fd_thresh=0.5, n_acompcor=5, use_gsr=0, low_pass=0.08, high_pass=0.008, min_vols=120):
    cmd = f"python {PIPE_PATH} --deriv-root {shlex.quote(deriv_root)} --atlas-img {shlex.quote(atlas_img)}"
    if atlas_labels:
        cmd += f" --atlas-labels {shlex.quote(atlas_labels)}"
    if participants:
        cmd += f" --participants {shlex.quote(participants)}"
    cmd += f" --out {shlex.quote(out)} --fd-thresh {fd_thresh} --n-acompcor {n_acompcor} --use-gsr {use_gsr}"
    cmd += f" --low-pass {low_pass} --high-pass {high_pass} --min-vols {min_vols}"
    output = run_command(cmd)
    return output

In [0]:
print(f"\n--- 4.1 Извлечение таймсерий для когорты подростков из {DERIV_DIR_TEEN} ---")
print(f"Таймсерии будут сохранены в {TS_OUT_TEEN}")

ret_ts_teen = run_pipeline(DERIV_DIR_TEEN, ATLAS_IMG, ATLAS_LABELS, TS_OUT_TEEN,
                           participants=PARTICIPANTS_FILE_TEEN, fd_thresh=FD_THRESH,
                           n_acompcor=N_ACOMPCOR, use_gsr=USE_GSR, low_pass=LOW_PASS, 
                           igh_pass=HIGH_PASS, min_vols=MIN_VOLS)

print("\nКод возврата:", ret_ts_teen)

In [ ]:
print(f"\n--- 4.2 Извлечение таймсерий для когорты взрослых из {DERIV_DIR_ADULT} ---")
print(f"Таймсерии будут сохранены в {TS_OUT_ADULT}")

ret_ts_adult = run_pipeline(DERIV_DIR_ADULT, ATLAS_IMG, ATLAS_LABELS, TS_OUT_ADULT,
                           participants=PARTICIPANTS_FILE_ADULT, fd_thresh=FD_THRESH,
                           n_acompcor=N_ACOMPCOR, use_gsr=USE_GSR, low_pass=LOW_PASS, 
                           high_pass=HIGH_PASS, min_vols=MIN_VOLS)

print("\nКод возврата:", ret_ts_adult)

### 5. QC / Preview
Гистограмма длин таймсерий после scrubbing + предпросмотр манифеста.

In [0]:
def analyze_timeseries(ts_out):
    """Анализирует извлечённые таймсерии в указанной директории."""
    manif = Path(ts_out) / 'processing_manifest.tsv'
    if manif.exists():
        dfm = pd.read_csv(manif, sep='\t')
        print("\nПервые строки processing_manifest.tsv:")
        print(dfm.head())
        if 'n_volumes_after_scrub' in dfm.columns:
            vals = dfm['n_volumes_after_scrub'].values
            plt.figure()
            plt.hist(vals, bins=20)
            plt.title('Длины таймсерий после scrubbing')
            plt.xlabel('Число объёмов')
            plt.ylabel('Кол-во запусков')
            plt.show()
    else:
        print('Не найден processing_manifest.tsv — проверь TS_OUT.')

In [ ]:
print(f"\n--- 5.1 Анализ таймсерий для когорты подростков в {TS_OUT_TEEN} ---")
analyze_timeseries(TS_OUT_TEEN)

In [ ]:
print(f"\n--- 5.2 Анализ таймсерий для когорты взрослых в {TS_OUT_ADULT} ---")
analyze_timeseries(TS_OUT_ADULT)